# Still in a rough state, need cleaner structures and encapsulations.

/scraper/src/cvh_scraper.py

In [ ]:
!pip install selenium webdriver_manager -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def scrape_data_collection_ids_for_offset(base_url, offset):
    """
    Scrapes data collection IDs from a given page based on the offset.

    Args:
    - base_url (str): The base URL to scrape, excluding the offset query parameter.
    - offset (int): The offset to apply to the base URL for pagination.

    Returns:
    - List of data collection IDs for the given offset (list).
    """
    # Set up Chrome options for headless execution
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Set up Selenium WebDriver with headless Chrome
    driver = webdriver.Chrome(options=chrome_options)

    data_collection_ids = []

    # Construct the URL with the provided offset
    url = f"{base_url}{offset}"

    # Open the URL
    driver.get(url)

    # Wait for the dynamic content to load
    time.sleep(5)  # Adjust the sleep time if necessary

    # Find all elements with the specified class
    rows = driver.find_elements(By.CLASS_NAME, 'spms-row')

    # Extract data-collection-id from each row
    for row in rows:
        data_collection_ids.append(row.get_attribute('data-collection-id'))

    # Close the WebDriver
    driver.quit()

    return data_collection_ids

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import concurrent
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import pandas as pd
import time
import os

def setup_driver(headless=True):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
    # service = Service(ChromeDriverManager().install())
    # driver = webdriver.Chrome(service=service, options=chrome_options)

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def wait_for_element(driver, by_method, value, timeout=10, retry_interval=3, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by_method, value)))
        except TimeoutException:
            # print(f"Retry {retries+1}/{max_retries} for element {value} after timeout.")
            time.sleep(retry_interval)
            retries += 1
        finally:
            # print(f"Retry {retries+1}/{max_retries} for element {value} after timeout.")
            time.sleep(retry_interval)
    # raise TimeoutException(f"Element {value} not found after {max_retries} retries.")
    return "Missing Value or Timeout"  # Return None if element is not found

def fetch_data(collection_id):
    driver = setup_driver()
    try:
        print(f"Fetching data for collection: {collection_id}\n.")
        driver.get(f"https://www.cvh.ac.cn/spms/detail.php?id={collection_id}")

        # Fetch specific details as per your logic
        data = {
            'Collection ID': collection_id,
            'Image Link': wait_for_element(driver, By.ID, "spm_image", max_retries=5).get_attribute('src'),
            'Phylum (门)': wait_for_element(driver, By.ID, "taxon_phy_c", max_retries=5).text,
            'Order (目)': wait_for_element(driver, By.ID, "taxon_ord_c", max_retries=5).text,
            'Family (科)': wait_for_element(driver, By.ID, "taxon_fam_c", max_retries=5).text,
            'Genus (属)': wait_for_element(driver, By.ID, "taxon_gen_c", max_retries=5).text,
            'Scientific Name': wait_for_element(driver, By.ID, "formattedName", max_retries=5).text,
            'Chinese Name': wait_for_element(driver, By.ID, "chineseName", max_retries=5).text,
            'Identified By': wait_for_element(driver, By.ID, "identifiedBy", max_retries=5).text,
            'Date Identified': wait_for_element(driver, By.ID, "dateIdentified", max_retries=5).text,
            'Collector': wait_for_element(driver, By.ID, "recordedBy", max_retries=5).text,
            'Collection Number': wait_for_element(driver, By.ID, "recordNumber", max_retries=5).text,
            'Collection Date': wait_for_element(driver, By.ID, "verbatimEventDate", max_retries=5).text,
            'Collection Location': wait_for_element(driver, By.ID, "locality", max_retries=5).text,
            'Altitude': wait_for_element(driver, By.ID, "elevation", max_retries=5).text,
            'Habitat': wait_for_element(driver, By.ID, "habitat", max_retries=5).text,
            'Phenology': wait_for_element(driver, By.ID, "reproductiveCondition", max_retries=5).text
        }
        return data
    finally:
        driver.quit()

def fetch_collection_data_concurrently(path, collection_ids, max_workers=10):
    results = []

    # Initialize an empty DataFrame to hold column names. This is useful if you expect your first few calls might fail and return None.
    name = next((cid for cid in collection_ids if cid is not None), "default")

    # Concatenate path and name to form temp_path
    temp_path = f"{path}/{name}.csv"
    print(temp_path)

    # Ensure the directory exists before attempting to write to it
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)

    pd.DataFrame(columns=['Collection ID', 'Image Link', 'Phylum (门)', 'Order (目)', 'Family (科)',
                          'Genus (属)', 'Scientific Name', 'Chinese Name', 'Identified By',
                          'Date Identified', 'Collector', 'Collection Number', 'Collection Date',
                          'Collection Location', 'Altitude', 'Habitat', 'Phenology']).to_csv(temp_path, mode='w', index=False, encoding='utf-8-sig')

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for cid in collection_ids:
            try:
                # Submitting the fetch_data function to the executor
                future = executor.submit(fetch_data, cid)
                futures.append(future)
            except Exception as submit_error:
                print(f"Failed to submit task for {cid}: {submit_error}")

        results = []
        for future in tqdm(as_completed(futures), total=len(collection_ids)):
            try:
                result = future.result()  # This will raise an exception if the task raised one
                if result:
                    # Attempt to append the result to the CSV file
                    try:
                        pd.DataFrame([result]).to_csv(temp_path, mode='a', header=False, index=False, encoding='utf-8-sig')
                    except PermissionError:
                        print(f"PermissionError: Unable to write to {temp_path}")
                    except Exception as write_error:
                        print(f"Error writing to file: {write_error}")
            except Exception as task_error:
                print(f"Error retrieving result from future: {task_error}")

    df = pd.DataFrame(results)
    return df

Total offset we have observed is:

299970, displaying 30 per page

In [ ]:
# Example usage:
base_url = "https://www.cvh.ac.cn/spms/list.php?&offset="
offset = 170000  # Example offset, can be adjusted or iterated in a loop
data_collection_ids = scrape_data_collection_ids_for_offset(base_url, offset)

In [ ]:
data_collection_ids

['e8570e26',
 'e8570eaf',
 'e8570f38',
 'e8570fbf',
 'e8571048',
 'e85710d4',
 'e8571162',
 'e85711e9',
 'e857126f',
 'e85712f6',
 'e857137e',
 'e8571406',
 'e857148c',
 'e8571511',
 'e8571596',
 'e857161b',
 'e85716ac',
 'e857173a',
 'e85717c1',
 'e8571849',
 'e85718d1',
 'e8571959',
 'e85719e0',
 '479e8b0c',
 'e8571aef',
 'e8571b75',
 'e8571bfb',
 'e8571c81',
 'e8571d0b',
 'e8571d97']

In [ ]:
def scape_cvh(max_offset=298000, total_samples=300, sample_size=30, results_path=".", check_path="./last_offset.txt"):
    base_url = "https://www.cvh.ac.cn/spms/list.php?&offset="
    max_offset = max_offset
    # total_samples = 10000
    total_samples = total_samples
    sample_size = sample_size
    results_path = results_path
    check_path = check_path

    # Load or initialize offset
    if os.path.exists(check_path):
        with open(check_path, 'r') as file:
            offset = int(file.read().strip())
    else:
        offset = max_offset

    data_collection = []

    for _ in range(total_samples // sample_size):
        # Update and wrap-around offset using modulo
        offset = (offset - sample_size) % max_offset
        print(f"Fetching data from offset: {offset}")

        # Scrape and fetch data
        data_collection_ids = scrape_data_collection_ids_for_offset(base_url, offset)
        results = fetch_collection_data_concurrently(results_path, data_collection_ids, sample_size)
        data_collection.extend(results)

        # Save the current offset
        with open(check_path, 'w') as file:
            file.write(str(offset))

    # Convert list to DataFrame
    df = pd.DataFrame(data_collection)
    df.to_csv(results_path + "/results.csv", header=True, index=False, encoding='utf-8-sig')
    print("Data collection and saving completed.")

In [ ]:
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"
check_path = "/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints/last_offset.txt"

with open(check_path, 'r') as file:
    max_offset = int(file.read().strip())

# or set it manually
# max_offset = 297870

# total_samples = 99000
total_samples = 6000

# sample size is always 30
scape_cvh(max_offset=int(max_offset), total_samples=total_samples, sample_size=30, results_path=g_path, check_path=check_path)

Fetching data from offset: 260560
/content/drive/MyDrive/Colab Notebooks/cs549_datasets/e91f750b.csv
Fetching data for collection: e91f79f9
.
Fetching data for collection: e91f7624
.


  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e91f7596
.
Fetching data for collection: e91f750b
.
Fetching data for collection: e91f76ba
.
Fetching data for collection: e91f7cb3
.
Fetching data for collection: e91f78e2
.
Fetching data for collection: e91f796e
.
Fetching data for collection: e91f7747
.
Fetching data for collection: e91f77cf
.
Fetching data for collection: e91f7d3c
.
Fetching data for collection: e91f7ed5
.
Fetching data for collection: e91f7859
.
Fetching data for collection: e91f7b99
.
Fetching data for collection: e91f7e4d
.
Fetching data for collection: e91f7a81
.
Fetching data for collection: e91f7c28
.
Fetching data for collection: e91f7dc4
.
Fetching data for collection: e91f7f5e
.
Fetching data for collection: e91f7fe6
.
Fetching data for collection: e91f7b0b
.
Fetching data for collection: e91f8298
.
Fetching data for collection: e91f84c3
.
Fetching data for collection: e91f80fb
.
Fetching data for collection: e91f8439
.
Fetching data for collection: e91f806e
.
Fetching data fo

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e91f699e
.
Fetching data for collection: e91f70a9
.
Fetching data for collection: e91f6dfb
.
Fetching data for collection: e91f6f95
.Fetching data for collection: e91f6ce7
.
Fetching data for collection: e91f6d72
.

Fetching data for collection: e91f6f0d
.
Fetching data for collection: e91f6e83
.
Fetching data for collection: e91f7258
.
Fetching data for collection: e91f713e
.
Fetching data for collection: e91f701e
.
Fetching data for collection: e91f71ce
.
Fetching data for collection: e91f736f
.
Fetching data for collection: e91f72e6
.
Fetching data for collection: e91f73f9
.
Fetching data for collection: e91f7481
.
Error retrieving result from future: 'str' object has no attribute 'text'
Error retrieving result from future: 'str' object has no attribute 'text'
Error retrieving result from future: 'str' object has no attribute 'text'
Error retrieving result from future: 'str' object has no attribute 'text'
Error retrieving result from future: 'str' objec

...

Fetching data for collection: e912d0f3
.
Fetching data for collection: e912d6df
.
Fetching data for collection: e912cfe0
.
Fetching data for collection: e912d76b
.
Fetching data for collection: e912d546
.
Fetching data for collection: e912dcd5
.
Fetching data for collection: e912d17c
.
Fetching data for collection: e912daa5
.
Fetching data for collection: e912d204
.
Fetching data for collection: e912d5ce
.
Fetching data for collection: e912d436
.
Fetching data for collection: e912d993
.
Fetching data for collection: e912d90a
.
Fetching data for collection: e912d882
.
Fetching data for collection: e912d4be
.
Fetching data for collection: e912d7fa
.
Fetching data for collection: e912d292
.
Fetching data for collection: e912d657
.
Fetching data for collection: e912da1c
.
Fetching data for collection: e912d3ac
.
Fetching data for collection: e912def8
.
Fetching data for collection: e912dde8
.
Fetching data for collection: e912dbb7
.
Fetching data for collection: e912db2f
.
Fetching data fo

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e912c2ff
.
Fetching data for collection: e912c498
.
Fetching data for collection: e912c7cd
.
Fetching data for collection: e912c96c
.
Fetching data for collection: e912c85c
.
Fetching data for collection: e912c5a6
.
Fetching data for collection: e912c62e
.
Fetching data for collection: e912c388
.
Fetching data for collection: e912cd29
.
Fetching data for collection: e912ce40
.
Fetching data for collection: e912c9f4
.
Fetching data for collection: e912cec8
.
Fetching data for collection: e912cb03
.
Fetching data for collection: e912cdb8
.
Fetching data for collection: e912cc14
.
Fetching data for collection: e912cf57
.
Fetching data for collection: e912cc9c
.
Fetching data for collection: e912ca7c
.
Fetching data for collection: e912cb8b
.
Fetching data for collection: e912c8e3
.
Fetching data from offset: 254590
/content/drive/MyDrive/Colab Notebooks/cs549_datasets/e912afa0.csv
Fetching data for collection: e912b366
.
Fetching data for collection: e912b027

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e912b145
.
Fetching data for collection: e912b0b5
.
Fetching data for collection: e912bbfc
.
Fetching data for collection: e912b256
.
Fetching data for collection: e912b3ed
.
Fetching data for collection: e912b475
.
Fetching data for collection: e912b6a1
.
Fetching data for collection: e912b585
.
Fetching data for collection: e912b1cd
.
Fetching data for collection: e912b72b
.
Fetching data for collection: e912b94d
.
Fetching data for collection: e912b2dd
.
Fetching data for collection: e912b7b5
.
Fetching data for collection: e912b8c5
.
Fetching data for collection: e912b4fd
.
Fetching data for collection: 47a01277
.
Fetching data for collection: e912b83d
.
Fetching data for collection: e912ba5d
.
Fetching data for collection: e912beae
.
Fetching data for collection: e912bc8a
.
Fetching data for collection: e912bb6e
.
Fetching data for collection: e912bd9c
.
Fetching data for collection: e912bae5
.
Fetching data for collection: e912bd14
.
Fetching data fo

In [ ]:
!pwd

# Check number of samples collected

In [ ]:
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"

In [ ]:
import pandas as pd
import os

# Define the path to the directory containing the CSV files
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(g_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append to the list, skip files with errors
for file in csv_files:
    file_path = os.path.join(g_path, file)
    try:
        df = pd.read_csv(file_path)
        if not df.empty:
            dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Skipped empty or invalid file: {file}")

# Concatenate all non-empty DataFrames
if dataframes:
    combined_df = pd.concat(dataframes)
    # Replace 'missing' with NaN for easier dropping of rows
    combined_df.replace('missing', pd.NA, inplace=True)
    # Drop rows where all elements are missing (NaN)
    combined_df.dropna(how='all', inplace=True)
    # Count the number of unique rows
    unique_rows_count = combined_df.drop_duplicates().shape[0]
    print(f'Number of unique rows: {unique_rows_count}')
else:
    print("No valid CSV files found.")

Skipped empty or invalid file: results.csv
Number of unique rows: 30913


In [ ]:
combined_df

In [ ]:
combined_df.nunique()

Collection ID          30913
Image Link             28681
Phylum (门)                 1
Order (目)                 69
Family (科)               218
Genus (属)               1251
Scientific Name         5458
Chinese Name            4488
Identified By            765
Date Identified         2353
Collector               2740
Collection Number      19471
Collection Date         7979
Collection Location       70
Altitude                2590
Habitat                 6875
Phenology                  6
dtype: int64

In [ ]:
import os

g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints"
file_path = os.path.join(g_path, 'results.csv')
combined_df.to_csv(file_path, index=False)

In [ ]:
file_path

'/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints/results.csv'